In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bikevshorse/bikeVShorse/Horses/horse1  (24).jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse68.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse75.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse57.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse36.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse67.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse12.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse59.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse38.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse33.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse81.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse29.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/h1.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse32.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse65.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse60.jpg
/kaggle/input/bikevshorse/bikeVShorse/Horses/horse8 .jpg
/kaggle/input/bikevshorse/bikeV

In [18]:
import torchvision.models as models
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [19]:
dataset = '../input/bikevshorse/bikeVShorse/'
import cv2
from PIL import Image
# Making Images and Labels for bike and Horse classification
Images = []
Labels = []

Classes = [cls for cls in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, cls))]
h_b = 0
count=0
for cls in Classes:

    dataset_Path = dataset+'/'+ cls
    files = [f for f in os.listdir(dataset_Path) if os.path.isfile(os.path.join(dataset_Path, f))]
    no_per_train=0
    for i in files:
        
        img_path = dataset_Path+'/'+i
        img = Image.open(img_path).convert('RGB')
        image = transform(img)
        Images.append(image)
        Labels.append(h_b)
        count=count+1
        
    h_b=h_b+1

In [20]:
from sklearn.model_selection import train_test_split
import numpy as np
train_Images,test_Images, train_Labels, test_Labels = train_test_split(Images, Labels, 
                                                train_size=0.8, random_state=42,shuffle = True,stratify = Labels)


In [21]:
# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

for param in alexnet.parameters():
    param.requires_grad = False
alexnet.classifier[6].requires_grad = True


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
import torch
train_Images = torch.stack(train_Images)
test_Images = torch.stack(test_Images)
train_features = alexnet.features(train_Images)
train_features = train_features.view(train_features.size(0), -1).detach().numpy()
test_features = alexnet.features(test_Images)
test_features = test_features.view(test_features.size(0), -1).detach().numpy()

In [23]:
from sklearn.linear_model import LogisticRegression


logistic_regression = LogisticRegression()
logistic_regression.fit(train_features, train_Labels)

from sklearn.metrics import accuracy_score

y_pred = logistic_regression.predict(test_features)
y_pred1 = logistic_regression.predict(train_features)
print("Training accuracy: ", accuracy_score(train_Labels,y_pred1))
print("Testing accuracy: ", accuracy_score(test_Labels,y_pred))

Training accuracy:  1.0
Testing accuracy:  1.0


In [24]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(train_features, train_Labels)

from sklearn.metrics import accuracy_score

y_pred = svc.predict(test_features)
y_pred1 = svc.predict(train_features)
print("Training accuracy: ", accuracy_score(train_Labels,y_pred1))
print("Testing accuracy: ", accuracy_score(test_Labels,y_pred))

Training accuracy:  1.0
Testing accuracy:  1.0


In [25]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(train_features, train_Labels)

from sklearn.metrics import accuracy_score

y_pred = model.predict(test_features)
y_pred1 = model.predict(train_features)
print("Training accuracy: ", accuracy_score(train_Labels,y_pred1))
print("Testing accuracy: ", accuracy_score(test_Labels,y_pred))

Training accuracy:  1.0
Testing accuracy:  0.75


In [28]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(train_features, train_Labels)

from sklearn.metrics import accuracy_score

y_pred = knn.predict(test_features)
y_pred1 = knn.predict(train_features)
print("Training accuracy: ", accuracy_score(train_Labels,y_pred1))
print("Testing accuracy: ", accuracy_score(test_Labels,y_pred))

Training accuracy:  0.986013986013986
Testing accuracy:  1.0
